In [ ]:
## imports
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, precision_score, f1_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from time import sleep
import pandas as pd
import numpy as np

# import custom modules
import sys
sys.path.insert(1, '../utils/')
import model_utils

In [ ]:
df = pd.read_csv("../../pre_processing/datasets/no_filtered_analysis.csv")

In [ ]:
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# Run all predictions based on fold n_splits
def run_prediction(X_train, X_test, y_train, y_test):

    # used to evaluate best model params
    gridsearch_metrics = {
            'REC':'recall', 
            'AUC':'roc_auc', 
            'Accuracy': 'accuracy'
    }

    # model gridsearch params
    model_params = {
        'criterion': ['gini', 'entropy'], 
        'splitter': ['best', 'random'],
        'max_depth': [5, 10, 15, 20],
        'max_leaf_nodes':[5,10,15,20,25],
        'max_features': ['auto', 'sqrt', 'log2', None]
    }

    # gridsearch initialization
    clf = GridSearchCV(DecisionTreeClassifier(),
                       param_grid = model_params,
                       scoring = gridsearch_metrics,
                       refit='Accuracy')

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    predict_data = {
        "confusion_matrix": confusion_matrix(y_test, predictions),
        "classification_report": classification_report(y_test, predictions),
        "model_accuracy": accuracy_score(y_test, predictions),
        "f1_score": f1_score(y_test, predictions),
        "precision_score": precision_score(y_test, predictions),
        "roc_score": roc_auc_score(y_test, predictions),
        "recall_score": recall_score(y_test, predictions),
        "epoch_params": clf.best_params_
    }

    return predict_data

In [ ]:
def main():
    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    skf = StratifiedKFold(n_splits = 4)
    skf.get_n_splits(X, y)
    scaler = StandardScaler()
    print(f"StratifiedKFold config: {skf} \n")
    all_predictions = []
    for train_index, test_index in skf.split(X, y):
        # Wait 20 seconds
        # sleep(20)
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # return best k element
        all_predictions.append(run_prediction(X_train, X_test, y_train, y_test))
    
    # General model outputs
    model_utils.generate_output(all_predictions)
    
    # CSV Output results
    model_utils.generate_output_csv(all_predictions)
    
    print("############################ FINISHED ############################")

In [ ]:
if __name__ == "__main__":
    main()